# Примеры оконных функций

In [1]:
import pandas as pd
from settings import db_config
from sqlalchemy import create_engine
pd.set_option("display.precision", 2)

In [2]:
# устанавливаем параметры
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config.USER, 
                                                         db_config.PWD, 
                                                         db_config.HOST, 
                                                         db_config.PORT,
                                                         db_config.DB)

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string)

In [4]:
def sql_request(query):
    """
    Функция запроса к базе данных и возврата ответа в виде pandas датафрейма

    query - запрос
    """

    return pd.read_sql(query, con = engine)

### Проранжировать расходы на привлечение пользователей за каждый день по убыванию

In [5]:
query = """
SELECT created_at::date,
       costs,
       DENSE_RANK() OVER (ORDER BY costs DESC)
FROM tools_shop.costs;
"""

In [6]:
sql_request(query)

,created_at,costs,dense_rank
0,2019-12-16,3247.58,1
1,2021-02-26,3121.92,2
2,2021-04-17,3113.96,3
3,2021-03-11,3087.05,4
4,2021-05-06,2934.81,5
...,...,...,...
1814,2016-07-06,1.99,1803
1815,2016-07-08,1.76,1804
1816,2016-10-25,1.59,1805
1817,2016-05-27,0.83,1806


### Список уникальных user_id пользователей, которые совершили три заказа и более

In [7]:
query = """
SELECT DISTINCT user_id
FROM 
    (SELECT user_id,
            ROW_NUMBER() OVER (PARTITION BY user_id) AS total_orders
     FROM tools_shop.orders
    ) AS orders
WHERE total_orders >= 3;
"""

In [8]:
sql_request(query)

,user_id
0,2950
1,3039
2,3763
3,7651
4,8468
...,...
77,247299
78,252685
79,253319
80,259210


### Количество заказов, в которых было четыре товара и более

In [9]:
query = """
SELECT COUNT(order_id)
FROM 
    (SELECT order_id,
            ROW_NUMBER() OVER (PARTITION BY order_id) AS items_count
     FROM tools_shop.order_x_item
    ) AS items
WHERE items_count >= 4;
"""

In [10]:
sql_request(query)

,count
0,4747


### Количество зарегистрированных пользователей по месяцам с накоплением

In [11]:
query = """
SELECT DISTINCT DATE_TRUNC('month', created_at)::date AS date,
       COUNT(user_id) OVER w
FROM tools_shop.users
WINDOW w AS (ORDER BY DATE_TRUNC('month', created_at)::date)
ORDER BY date;
"""

In [12]:
sql_request(query)

,date,count
0,2016-02-01,9
1,2016-03-01,162
2,2016-04-01,450
3,2016-05-01,826
4,2016-06-01,1307
...,...,...
60,2021-02-01,108502
61,2021-03-01,111420
62,2021-04-01,114203
63,2021-05-01,117082


### Сумма трат на привлечение пользователей с накоплением по месяцам c 2017 по 2018 год

In [13]:
query = """
SELECT DISTINCT DATE_TRUNC('month', created_at)::date,
       SUM(costs) OVER w
FROM tools_shop.costs
WHERE EXTRACT(YEAR FROM created_at) BETWEEN 2017 AND 2018
WINDOW w AS (ORDER BY DATE_TRUNC('month', created_at)::date);
"""

In [14]:
sql_request(query)

,date_trunc,sum
0,2017-01-01,8054.75
1,2017-02-01,16472.46
2,2017-03-01,26315.39
3,2017-04-01,40632.33
4,2017-05-01,53851.92
5,2017-06-01,69186.32
6,2017-07-01,84158.57
7,2017-08-01,99067.96
8,2017-09-01,113845.30
9,2017-10-01,129929.17


### Количество событий view_item по месяцам с накоплением только для тех пользователей, которые совершили хотя бы одну покупку

In [15]:
query = """
WITH events AS
  (SELECT DATE_TRUNC('month', event_time)::date AS event_month,
          COUNT(DISTINCT event_id) AS events_cnt
   FROM tools_shop.events e
   JOIN tools_shop.orders o ON e.user_id = o.user_id
   WHERE event_name = 'view_item'
   GROUP BY event_month)
SELECT event_month,
       events_cnt,
       SUM(events_cnt) OVER (ORDER BY event_month) AS cum_sum
FROM events;
"""

In [16]:
sql_request(query)

,event_month,events_cnt,cum_sum
0,2016-03-01,10,10.0
1,2016-04-01,23,33.0
2,2016-05-01,84,117.0
3,2016-06-01,111,228.0
4,2016-07-01,190,418.0
...,...,...,...
59,2021-02-01,1966,75857.0
60,2021-03-01,2059,77916.0
61,2021-04-01,1824,79740.0
62,2021-05-01,1701,81441.0


### Сумма трат на привлечение пользователей по месяцам, а также разница в тратах между текущим и предыдущим месяцами

In [17]:
query = """
WITH costs_month AS (
    SELECT DATE_TRUNC('month', created_at)::date AS dt,
           SUM (costs) AS costs_sum
    FROM tools_shop.costs
    GROUP BY DATE_TRUNC('month', created_at)::date 
)

SELECT *, 
       costs_sum - LAG(costs_sum, 1, costs_sum) OVER (ORDER BY dt) AS diff
FROM costs_month;
"""

In [18]:
sql_request(query)

,dt,costs_sum,diff
0,2016-03-01,130.59,0.00
1,2016-04-01,157.17,26.58
2,2016-05-01,1006.73,849.56
3,2016-06-01,1979.74,973.01
4,2016-07-01,4021.09,2041.35
...,...,...,...
59,2021-02-01,37628.07,-1375.53
60,2021-03-01,38275.16,647.09
61,2021-04-01,36832.95,-1442.21
62,2021-05-01,36974.49,141.54


### Сумма выручки по годам и разница выручки между текущим и следующим годом

In [19]:
query = """
WITH revenue_by_year AS (
    SELECT DATE_TRUNC('year', created_at)::date AS dt, 
           SUM(total_amt) AS yr_revenue
    FROM tools_shop.orders
    GROUP BY dt
) 

SELECT dt,
       yr_revenue,
       LEAD(yr_revenue, 1, yr_revenue) OVER (ORDER BY dt) - yr_revenue AS next_year_revenue
FROM revenue_by_year;
"""

In [20]:
sql_request(query)

,dt,yr_revenue,next_year_revenue
0,2016-01-01,1.37e+05,428459.10
1,2017-01-01,5.65e+05,339705.02
2,2018-01-01,9.05e+05,253992.89
3,2019-01-01,1.16e+06,268032.02
4,2020-01-01,1.43e+06,-793519.36
5,2021-01-01,6.33e+05,0.00


In [21]:
engine.dispose()